# OpenAI Function Calling - Coding Practice Notebook 

#### Using the Function Calling feature gives your chatbot super powers.
#### What are these super powers you ask? Ok, let me tell you ....
(a) Describe the function to the model so that it can then

(b) Output JSON object containing arguments to those calls based on the user query

This feature allows for a more reliable method for your app/bot, to be able to interact with various external tools. This is a huge win, to be able to do since it it cuts short a whole bunch of ReAct(or other forms of prompt engineering). This is possible because OpenAI has fine tuned the models to detect when a function needs to be called  and then respond with it's corresponding JSON matching the funtions signature.

In this notebook we explore the function calling feature by building a simple weather bot to provide us with the current weather.


### This is the format let the OpenAI models know, the function format
One or more function definition in a JSON format

In [3]:

# OpenAI function format: https://openai.com/blog/function-calling-and-other-api-updates

function_definitions = [
        {
            "name": "get_current_weather",
                            "description": "Get the current weather in a given location",
                            "parameters": {
                                "type": "object",
                                "properties": {
                                    "location": {
                                        "type": "string",
                                        "description": "The city and state, e.g. San Francisco, CA",
                                    },
                                    "unit": {
                                        "type": "string",
                                        "description": "The temperature unit to use. Infer this from the users location.",
                                        "enum": ["celsius", "fahrenheit"]
                                    },
                                },
                                "required": ["location", "unit"],
                    },
        }

    ]


In [ ]:
# Make sure we get the latest langchain with the functions calling feature updates

!pip install -U langchain

In [4]:
# Import Dependencies

# For Models
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# For Agent and Tools
from langchain.agents import AgentType
from langchain.agents import initialize_agent, Tool
from langchain.tools import BaseTool, format_tool_to_openai_function
from pydantic import BaseModel, Field
from typing import Optional, Type
import json

# For Message schemas, 
from langchain.schema import HumanMessage, AIMessage, ChatMessage, FunctionMessage


In [2]:
# Import OpenAI key

import os
os.environ["OPENAI_API_KEY"] = ""
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Set model name for this notebook: gpt-3.5-turbo-0613 or gpt-4-0613 or above 

model_name = "gpt-3.5-turbo-0613"


#### Simple Case

We are going to use function calling in conjunction with our model, for figuring out the weather in NYC

#### Step 1: Write a custom function

In this case I have mocked the response data. But in theory one could call an external api to gather this information and return a response.

In [5]:
# Lets define a function/tool for getting the weather. In this demo it we mockthe output
# In real life, you'd end up calling a library/API such as PWOWM (open weather map) library: 
# Depending on your app's functionality, you may also, call vendor/external or internal custom APIs

def get_current_weather(location, unit):
    
    # Call an external API to get relevant information (like serpapi, etc)
    # Here for the demo we will send a mock response
    weather_info = {
        "location": location,
        "temperature": "78",
        "unit": unit,
        "forecast": ["sunny", "with a chance of meatballs"],
    }
    return weather_info



Quick test to make sure our function works

In [7]:
test_location = "NYC"
test_unit = "Fahrenheit"

function_response = get_current_weather(
    test_location,
    test_unit
)

function_response

{'location': 'NYC',
 'temperature': '78',
 'unit': 'Fahrenheit',
 'forecast': ['sunny', 'with a chance of meatballs']}

#### Step 2: Create a custom tool that uses our previously defined get_current_weather function

Define a pydantic class GetCurrentWeatherCheckInput to define the function arguments that the model should return. 
Then define the tool while passing the pydantic our previously defined GetCurrentWeatherCheckInput to the tool as argument schema

In [9]:
class GetCurrentWeatherCheckInput(BaseModel):
    # Check the input for Weather
    location: str = Field(..., description = "The name of the location name for which we need to find the weather")
    unit: str = Field(..., description = "The unit for the temperature value")

In [10]:
class GetCurrentWeatherTool(BaseTool):
    
    name = "get_current_weather"
    description = "Used to find the weather for a given location in said unit"
    
    def _run(self, location: str, unit: str):
        #print("I am running!")
        weather_response = get_current_weather(location, unit)
        return weather_response
    
    def _arun(self, location: str, unit:str):
        raise NotImplementedError("This tool does not support async")
   
    args_schema: Optional[Type[BaseModel]] = GetCurrentWeatherCheckInput


#### Step 3: Setup the the tool list as well as the functions list

Note: format_tool_to_openai_function is a Langcahin function to convert the tool to the function format expected by the model

In [11]:
tools = [GetCurrentWeatherTool()]
functions = [format_tool_to_openai_function(tool_name) for tool_name in tools]

In [12]:
tools[0]

GetCurrentWeatherTool(name='get_current_weather', description='Used to find the weather for a given location in said unit', args_schema=<class '__main__.GetCurrentWeatherCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False)

In [13]:
functions

[{'name': 'get_current_weather',
  'description': 'Used to find the weather for a given location in said unit',
  'parameters': {'title': 'GetCurrentWeatherCheckInput',
   'type': 'object',
   'properties': {'location': {'title': 'Location',
     'description': 'The name of the location name for which we need to find the weather',
     'type': 'string'},
    'unit': {'title': 'Unit',
     'description': 'The unit for the temperature value',
     'type': 'string'}},
   'required': ['location', 'unit']}}]

#### Step 4: Have the model output back to us the name of the function with its argument 


In [14]:
# Let's imagine that the user lives in NYC
query = "What is the temperature like in NYC?"

In [15]:
# Let's first create the chat model
model = ChatOpenAI(
            model = model_name, 
            temperature = 0,
        )

response_ai_message = model.predict_messages([HumanMessage(content=query)], functions=functions)

response_ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "New York",\n  "unit": "Celsius"\n}'}}, example=False)

In [16]:
type(response_ai_message)

langchain.schema.AIMessage

In [17]:
response_ai_message.additional_kwargs['function_call']

{'name': 'get_current_weather',
 'arguments': '{\n  "location": "New York",\n  "unit": "Celsius"\n}'}

In [18]:
import json
_args = json.loads(response_ai_message.additional_kwargs['function_call'].get('arguments'))
_args

{'location': 'New York', 'unit': 'Celsius'}

In [19]:
tools[0]

GetCurrentWeatherTool(name='get_current_weather', description='Used to find the weather for a given location in said unit', args_schema=<class '__main__.GetCurrentWeatherCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False)

#### Step 5: Run the inferred tool with the inferred arguments

In [20]:
tool_result = tools[0](_args)
tool_result

{'location': 'New York',
 'temperature': '78',
 'unit': 'Celsius',
 'forecast': ['sunny', 'with a chance of meatballs']}

#### Step 6: Set FunctionMessage 

We do this inorder to get the final response from the model in natural language

In [ ]:
FunctionMessage(name = "get_current_weather", content=str(tool_result))
             

#### Step 7: Get the final response from the model

In [21]:
response_final = model.predict_messages(
                    [
                        HumanMessage(content= query),
                        response_ai_message,
                        FunctionMessage(name='get_current_weather',content=str(tool_result)),
                    ], 
                    functions=functions
                )
response_final

AIMessage(content='The temperature in NYC is currently 78 degrees Celsius. The weather forecast is sunny, with a chance of meatballs.', additional_kwargs={}, example=False)

### Getting a similar result using Agent

#### Set up the model

In [22]:

model_using_agent = ChatOpenAI(temperature = 0, model = model_name)


#### Initailize the agent, pass the tools

In [23]:
agent_for_function = initialize_agent(
                        tools,
                        model_using_agent,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True
                    )

#### Run the agent

In [24]:
agent_for_function.run(query)



> Entering new  chain...

Invoking: `get_current_weather` with `{'location': 'New York', 'unit': 'celsius'}`


{'location': 'New York', 'temperature': '78', 'unit': 'celsius', 'forecast': ['sunny', 'with a chance of meatballs']}The current temperature in New York City is 78 degrees Celsius. The weather forecast for today is sunny, with a chance of meatballs.

> Finished chain.


'The current temperature in New York City is 78 degrees Celsius. The weather forecast for today is sunny, with a chance of meatballs.'